In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import gc
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

SAMPLE_SIZE = 0.5
%matplotlib inline

In [ ]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisosONEHOT.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantesONEHOT.csv")

avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos_normales = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
avisos_normales = avisos_normales.loc[:,["nombre_area","idaviso","tipo_de_trabajo","nivel_laboral"]]

vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [ ]:

avisos_normales.head()
    

In [ ]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

In [ ]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

In [ ]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')
vistas3.head()

In [ ]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

In [ ]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})
vistas_final = pd.merge(vistas3,avisos_normales, on= "idaviso", how = "left")
vistas_final.head()

In [ ]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")
postulaciones=postulaciones.append(no_postulaciones, ignore_index=True)
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
postulaciones = postulaciones.sample(int(round(len(postulaciones)*SAMPLE_SIZE)))
no_postulaciones = 0
gc.collect()

In [ ]:
vistas_final["vistas_por_area"]=0
vistas_final["vistas_por_tipo"]=0
vistas_final["vistas_por_nivel"]=0

vistas_area = vistas_final.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_tipo= vistas_final.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_nivel=vistas_final.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})

vistas_final = 0
gc.collect()

In [ ]:
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [ ]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones = pd.merge(postulaciones,avisos_normales, on = "idaviso", how = "left")
postulaciones.drop("nombre_area_y", axis = 1, inplace=True)
postulaciones.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones["vistas_al_aviso"] = postulaciones["vistas_al_aviso"].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.cantidad = postulaciones.cantidad.fillna(0)
postulaciones.vistas_postulante = postulaciones.vistas_postulante.fillna(0)

postulaciones.sample(20)

In [ ]:
postulaciones_area = postulaciones.groupby(['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_tipo= postulaciones.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_nivel =postulaciones.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_area=postulaciones_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_tipo=postulaciones_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_nivel=postulaciones_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})

In [ ]:
postulaciones.dropna(axis = 1, inplace=True)

In [15]:
postulaciones_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Cadetería,1
1,0z5Dmrd,Call Center,1
2,0z5Dmrd,Comercial,1
3,0z5Dmrd,Facturación,1
4,0z5Dmrd,Ingeniería Mecánica,1


In [16]:
postulaciones=pd.merge(postulaciones,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')

postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.drop(["tipo_de_trabajo","nivel_laboral"], axis = 1, inplace=True)



In [17]:
#mean encoding
area_mean = postulaciones.groupby("nombre_area").agg({"sepostulo":"mean"})
area_mean.head()

means = area_mean.to_dict()["sepostulo"]
postulaciones.nombre_area = postulaciones.nombre_area.replace(means)
postulaciones.sample(10)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
2709359,1112337841,MVPX0J3,1,25.0,False,True,False,False,False,False,...,21.0,0.0,0.000000,6.0,18.0,18.0,False,6,31,16
5269438,1112225360,xkdaj4a,0,38.0,True,False,False,False,False,False,...,1.0,0.0,0.000000,0.0,1.0,1.0,False,0,5,4
2210334,1112389521,a4q060,1,49.0,False,True,False,False,False,False,...,22.0,0.0,0.000000,9.0,16.0,16.0,False,10,26,12
332269,1111709950,dYolOGY,1,34.0,False,True,False,False,False,False,...,187.0,1.0,0.005348,25.0,53.0,53.0,True,8,19,43
5764746,1112460526,YKXeBY,0,31.0,False,True,False,False,False,False,...,0.0,0.0,0.000000,0.0,0.0,0.0,False,1,9,7
1699797,1112445738,ZDaxB0V,1,27.0,True,False,False,False,False,False,...,214.0,1.0,0.004673,3.0,125.0,125.0,True,0,29,5
5179194,1112425230,8mJk1O,0,25.0,False,True,False,False,False,False,...,4.0,0.0,0.000000,0.0,3.0,3.0,False,0,9,5
3433790,1112246980,ekO0Kb2,1,23.0,False,True,False,False,False,False,...,4.0,0.0,0.000000,0.0,2.0,2.0,False,1,13,15
4786155,1112286159,ebe8e8,1,29.0,True,False,False,False,False,False,...,2.0,0.0,0.000000,1.0,2.0,2.0,False,3,11,7
2285063,1002490173,e9AzDj,1,37.0,True,False,False,False,False,False,...,73.0,1.0,0.013699,22.0,59.0,59.0,True,21,45,31


In [18]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')


x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

postulaciones=0

gc.collect()

Train:  5754287 Test:  639366


91

In [19]:
model = XGBClassifier(objective='binary:logistic',max_depth=18,learning_rate=0.1,subsample=0.8,colsample_bytree=0.8,n_estimators=1200,silent=False,n_jobs=-1,verbose=1)
dt=model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=5)

[0]	validation_0-error:0.112393
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.104934
[2]	validation_0-error:0.102566
[3]	validation_0-error:0.101554
[4]	validation_0-error:0.101253
[5]	validation_0-error:0.100102
[6]	validation_0-error:0.09963
[7]	validation_0-error:0.099305
[8]	validation_0-error:0.099075
[9]	validation_0-error:0.098616
[10]	validation_0-error:0.097955
[11]	validation_0-error:0.097683
[12]	validation_0-error:0.097099
[13]	validation_0-error:0.096677
[14]	validation_0-error:0.096419
[15]	validation_0-error:0.095954
[16]	validation_0-error:0.095897
[17]	validation_0-error:0.095524
[18]	validation_0-error:0.095252
[19]	validation_0-error:0.095161
[20]	validation_0-error:0.094514
[21]	validation_0-error:0.094201
[22]	validation_0-error:0.094049
[23]	validation_0-error:0.093962
[24]	validation_0-error:0.093558
[25]	validation_0-error:0.093269
[26]	validation_0-error:0.092895
[27]	validation_0-error:0.092459
[28]	validation_0-erro

In [20]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=model.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9782281601709464


In [21]:
print("dale mono")

dale mono


In [22]:
joblib.dump(model, "Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean3.pkl")

['Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean2.pkl']

NameError: name 'prediccion' is not defined

In [24]:
prediccion = pd.read_csv("Data/fiuba_entrenamiento/test_final_100k.csv")
prediccion = pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
prediccion = pd.merge(prediccion,avisos,on='idaviso',how='inner')
prediccion = pd.merge(prediccion,avisos_normales,on='idaviso',how='inner')
prediccion.drop("nombre_area_y", axis = 1, inplace=True)
prediccion.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)

prediccion = pd.merge(prediccion,vistas,on='idpostulante',how='left')
prediccion.sample(10)


prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion['cantidad']=prediccion['cantidad'].fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion['lo_vio']=(prediccion['vistas_al_aviso']>0)

prediccion=pd.merge(prediccion,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion=pd.merge(prediccion,vistas_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.nombre_area = prediccion.nombre_area.replace(means)
prediccion.sample(10)

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
80345,64403,1112458199,5q1Gkv,36.0,False,True,False,False,False,False,...,4.0,0.0,0.000000,False,0.0,11.0,0.0,0.0,3.0,3.0
10037,69026,1112460989,xkaeNe6,26.0,False,True,False,False,False,False,...,33.0,1.0,0.030303,True,0.0,14.0,2.0,1.0,23.0,23.0
84898,33689,1112404095,Z4X6xY,40.0,False,True,False,False,False,False,...,2.0,0.0,0.000000,False,0.0,7.0,3.0,0.0,2.0,2.0
44513,53787,1112445484,EOeOl8,34.0,False,True,False,False,False,False,...,2.0,1.0,0.500000,True,0.0,0.0,2.0,2.0,1.0,1.0
41694,19039,1112221462,8MPdJEO,27.0,False,True,False,False,False,False,...,14.0,1.0,0.071429,True,0.0,9.0,7.0,2.0,14.0,14.0
51626,42483,1112423499,aEE8BW,34.0,False,True,False,False,False,False,...,0.0,0.0,0.000000,False,0.0,5.0,4.0,0.0,0.0,0.0
796,73849,1112463495,EzO8o96,24.0,False,True,False,False,False,False,...,1.0,0.0,0.000000,False,7.0,63.0,44.0,0.0,1.0,1.0
41826,37753,1112414416,BmDXQlz,24.0,False,True,False,False,False,False,...,213.0,1.0,0.004695,True,36.0,145.0,8.0,46.0,166.0,166.0
31469,1374,1110669632,kPjLRR1,35.0,False,True,False,False,False,False,...,0.0,0.0,0.000000,False,1.0,5.0,0.0,0.0,0.0,0.0
86817,38602,1112415596,8LA0LR,37.0,False,True,False,False,False,False,...,379.0,1.0,0.002639,True,4.0,15.0,11.0,66.0,329.0,329.0


In [25]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,26.0,0.0,0.0,False,0.0,6.0,0.0,3.0,20.0,20.0
1,1,739260,6v1xdL,30.0,False,True,False,False,False,False,...,4.0,0.0,0.0,False,2.0,33.0,1.0,0.0,3.0,3.0
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,10.0,0.0,0.0,False,1.0,4.0,1.0,1.0,6.0,6.0
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,0.0,False,0.0,7.0,1.0,0.0,0.0,0.0
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,1.0,0.0,0.0,False,0.0,6.0,0.0,0.0,1.0,1.0


In [26]:
x_final=prediccion[features]
y_final=model.predict_proba(x_final)
y_final2=[]
print(model.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

prediccion['sepostulo']=y_final2
prediccion.head()


[0 1]
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,0.0,0.0,False,0.0,6.0,0.0,3.0,20.0,20.0,0.100518
1,1,739260,6v1xdL,30.0,False,True,False,False,False,False,...,0.0,0.0,False,2.0,33.0,1.0,0.0,3.0,3.0,0.257462
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,0.0,0.0,False,1.0,4.0,1.0,1.0,6.0,6.0,0.022451
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,False,0.0,7.0,1.0,0.0,0.0,0.0,0.010519
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,0.0,0.0,False,0.0,6.0,0.0,0.0,1.0,1.0,0.002036


In [27]:
prediccion=prediccion[['id','sepostulo']]
print(prediccion.sepostulo.mean())
prediccion.head()

0.49329292357781124


,id,sepostulo
0,0,0.100518
1,1,0.257462
2,2,0.022451
3,3,0.010519
4,4,0.002036


In [28]:
prediccion.to_csv("Data/fiuba_entrenamiento/pocho/predicciones/xgbmean3.csv",index=False)